In [0]:
%sql

CREATE TABLE IF NOT EXISTS fake_records (name STRING, home_address STRING);

ALTER TABLE fake_records
SET TBLPROPERTIES (
    'delta.minReaderVersion' = '2',
    'delta.minWriterVersion' = '5',
    'delta.columnMapping.mode' = 'name'
  )

In [0]:
bronzeDF = (spark.readStream 
              .format("cloudFiles") 
              .option("cloudFiles.format", "csv") 
              .option("cloudFiles.maxFilesPerTrigger", 1) 
              .option("header", "true")
              .schema("name string, home_address string")
              .load('/tmp/fake_records/raw/') 
             .writeStream 
               .format("delta") 
               .trigger(once=True) 
               .option("checkpointLocation", "/tmp/fake_records/checkpoint/") 
               .table("fake_records")
           )

In [0]:
%sql 
DESCRIBE EXTENDED fake_records

In [0]:
%sql

SELECT *
FROM fake_records

In [0]:
%sql 

DROP TABLE fake_records

In [0]:
%fs

rm -r /tmp/fake_records/schema

In [0]:
%fs

rm -r /tmp/fake_records/checkpoint


In [0]:
%fs

ls /tmp/fake_records/schema